In [1]:
# load original skempi_v2.csv
import pandas as pd

raw_df = pd.read_csv("./datasets/skempi_v2/skempi_v2.csv", sep=';')
raw_df = raw_df[['#Pdb', 'Mutation(s)_cleaned', 'Affinity_wt_parsed', 'Affinity_mut_parsed', 'Temperature']]
col_mapping = {'#Pdb': 'pdb',
               'Mutation(s)_cleaned': 'mutation', 
               'Affinity_wt_parsed': 'affinity_wt', 
               'Affinity_mut_parsed': 'affinity_mut',
               "Temperature": "temp"}
raw_df.rename(columns=col_mapping, inplace=True)
raw_df

,pdb,mutation,affinity_wt,affinity_mut,temp
0,1CSE_E_I,LI38G,1.120000e-12,5.260000e-11,294
1,1CSE_E_I,LI38S,1.120000e-12,8.330000e-12,294
2,1CSE_E_I,LI38P,1.120000e-12,1.020000e-07,294
3,1CSE_E_I,LI38I,1.120000e-12,1.720000e-10,294
4,1CSE_E_I,LI38D,1.120000e-12,1.920000e-09,294
...,...,...,...,...,...
7080,3QIB_ABP_CD,KP8R,5.500000e-06,2.400000e-04,298
7081,3QIB_ABP_CD,TP11A,5.500000e-06,1.100000e-03,298
7082,3QIB_ABP_CD,TP11S,5.500000e-06,3.380000e-05,298
7083,3QIB_ABP_CD,TP11N,5.500000e-06,4.340000e-05,298


In [2]:
# compute ddg & daffinity
import numpy as np
df = raw_df
df['temp'] = df['temp'].map(lambda temp: float(temp[:3]) if isinstance(temp, str) else temp)

R = float(8.314/4184)
df['ddg'] = R * df['temp'] * (np.log(df['affinity_mut']) - np.log(df['affinity_wt']))

df['daffinity'] = df['affinity_mut'] - df['affinity_wt']

# df = df[['pdb', 'mutation', 'ddg', 'daffinity', 'temp']]
df

,pdb,mutation,affinity_wt,affinity_mut,temp,ddg,daffinity
0,1CSE_E_I,LI38G,1.120000e-12,5.260000e-11,294.0,2.248833,5.148000e-11
1,1CSE_E_I,LI38S,1.120000e-12,8.330000e-12,294.0,1.172229,7.210000e-12
2,1CSE_E_I,LI38P,1.120000e-12,1.020000e-07,294.0,6.671276,1.019989e-07
3,1CSE_E_I,LI38I,1.120000e-12,1.720000e-10,294.0,2.940988,1.708800e-10
4,1CSE_E_I,LI38D,1.120000e-12,1.920000e-09,294.0,4.350434,1.918880e-09
...,...,...,...,...,...,...,...
7080,3QIB_ABP_CD,KP8R,5.500000e-06,2.400000e-04,298.0,2.235909,2.345000e-04
7081,3QIB_ABP_CD,TP11A,5.500000e-06,1.100000e-03,298.0,3.137419,1.094500e-03
7082,3QIB_ABP_CD,TP11S,5.500000e-06,3.380000e-05,298.0,1.075181,2.830000e-05
7083,3QIB_ABP_CD,TP11N,5.500000e-06,4.340000e-05,298.0,1.223219,3.790000e-05


In [3]:
# remove repeated wt-mut data
dataset = {}
for i in range(len(df)):
    wt = df.loc[i, 'pdb']
    mut = df.loc[i, 'mutation']
    if (wt, mut) not in dataset:
        dataset[(wt, mut)] = []
    dataset[(wt, mut)].append(i)
print(f"Num of wt-mut pair: {len(dataset)}")

drop_list = []
for wt_mut_pair in dataset:
    repeat_idxs = dataset[wt_mut_pair]
    if len(repeat_idxs) == 1:
        continue
    idx_298 = []
    for idx in repeat_idxs:
        if df.loc[idx, 'temp'] == 298:
            idx_298.append(idx)
    if len(idx_298) == 0:
        idx_298 = repeat_idxs
    
    ddg = np.mean([df.loc[idx, 'ddg'] for idx in idx_298])
    mean_daffinity = np.mean([df.loc[idx, 'daffinity'] for idx in idx_298])
    df.loc[repeat_idxs[0], 'ddg'] = ddg
    df.loc[repeat_idxs[0], 'daffinity'] = mean_daffinity
    
    drop_list += repeat_idxs[1:]

print(f"Num of redundant data: {len(drop_list)}")
df = df.drop(index=drop_list)
df.reset_index(drop=True, inplace=True)
# df = df[['pdb', 'mutation', 'ddg', 'daffinity']]
df

Num of wt-mut pair: 6193
Num of redundant data: 892


,pdb,mutation,affinity_wt,affinity_mut,temp,ddg,daffinity
0,1CSE_E_I,LI38G,1.120000e-12,5.260000e-11,294.0,2.248833,5.148000e-11
1,1CSE_E_I,LI38S,1.120000e-12,8.330000e-12,294.0,1.172229,7.210000e-12
2,1CSE_E_I,LI38P,1.120000e-12,1.020000e-07,294.0,6.671276,1.019989e-07
3,1CSE_E_I,LI38I,1.120000e-12,1.720000e-10,294.0,2.940988,1.708800e-10
4,1CSE_E_I,LI38D,1.120000e-12,1.920000e-09,294.0,4.350434,1.918880e-09
...,...,...,...,...,...,...,...
6188,3QIB_ABP_CD,KP8R,5.500000e-06,2.400000e-04,298.0,2.235909,2.345000e-04
6189,3QIB_ABP_CD,TP11A,5.500000e-06,1.100000e-03,298.0,3.137419,1.094500e-03
6190,3QIB_ABP_CD,TP11S,5.500000e-06,3.380000e-05,298.0,1.075181,2.830000e-05
6191,3QIB_ABP_CD,TP11N,5.500000e-06,4.340000e-05,298.0,1.223219,3.790000e-05


In [4]:
# remove data that EvoEF2 failed to generate
import os

mut_pdb_path = "./datasets/skempi_v2/mut_PDBs/"

drop_list = []
for idx in range(len(df)):
    wt = df.loc[idx, 'pdb']
    mut = df.loc[idx, 'mutation']
    mut_pdb_filename = wt + '_' + mut + '.pdb'
    if not os.path.exists(mut_pdb_path + mut_pdb_filename):
        drop_list.append(idx)
        
print(f"Num of EvoEF2-fail-to-generate data: {len(drop_list)}") 
df = df.drop(index=drop_list)
df.reset_index(drop=True, inplace=True)
df

Num of EvoEF2-fail-to-generate data: 93


,pdb,mutation,affinity_wt,affinity_mut,temp,ddg,daffinity
0,1CSE_E_I,LI38G,1.120000e-12,5.260000e-11,294.0,2.248833,5.148000e-11
1,1CSE_E_I,LI38S,1.120000e-12,8.330000e-12,294.0,1.172229,7.210000e-12
2,1CSE_E_I,LI38P,1.120000e-12,1.020000e-07,294.0,6.671276,1.019989e-07
3,1CSE_E_I,LI38I,1.120000e-12,1.720000e-10,294.0,2.940988,1.708800e-10
4,1CSE_E_I,LI38D,1.120000e-12,1.920000e-09,294.0,4.350434,1.918880e-09
...,...,...,...,...,...,...,...
6095,3QIB_ABP_CD,KP8R,5.500000e-06,2.400000e-04,298.0,2.235909,2.345000e-04
6096,3QIB_ABP_CD,TP11A,5.500000e-06,1.100000e-03,298.0,3.137419,1.094500e-03
6097,3QIB_ABP_CD,TP11S,5.500000e-06,3.380000e-05,298.0,1.075181,2.830000e-05
6098,3QIB_ABP_CD,TP11N,5.500000e-06,4.340000e-05,298.0,1.223219,3.790000e-05


In [5]:
# remove NaN data
df = df.dropna()
df.reset_index(drop=True, inplace=True)
df

,pdb,mutation,affinity_wt,affinity_mut,temp,ddg,daffinity
0,1CSE_E_I,LI38G,1.120000e-12,5.260000e-11,294.0,2.248833,5.148000e-11
1,1CSE_E_I,LI38S,1.120000e-12,8.330000e-12,294.0,1.172229,7.210000e-12
2,1CSE_E_I,LI38P,1.120000e-12,1.020000e-07,294.0,6.671276,1.019989e-07
3,1CSE_E_I,LI38I,1.120000e-12,1.720000e-10,294.0,2.940988,1.708800e-10
4,1CSE_E_I,LI38D,1.120000e-12,1.920000e-09,294.0,4.350434,1.918880e-09
...,...,...,...,...,...,...,...
5830,3QIB_ABP_CD,KP8R,5.500000e-06,2.400000e-04,298.0,2.235909,2.345000e-04
5831,3QIB_ABP_CD,TP11A,5.500000e-06,1.100000e-03,298.0,3.137419,1.094500e-03
5832,3QIB_ABP_CD,TP11S,5.500000e-06,3.380000e-05,298.0,1.075181,2.830000e-05
5833,3QIB_ABP_CD,TP11N,5.500000e-06,4.340000e-05,298.0,1.223219,3.790000e-05


In [6]:
print(f"daffinity range: [{df['daffinity'].min()}, {df['daffinity'].max()}]")
print(f"ddg range: [{df['ddg'].min()}, {df['ddg'].max()}]")


daffinity range: [-0.000111, 0.04049885]
ddg range: [-12.221723141911879, 12.221723141911879]


In [ ]:
df.iloc[[df['daffinity'].idxmin()]]

In [ ]:
df.iloc[[df['daffinity'].idxmax()]]

In [ ]:
df.iloc[[df['ddg'].idxmin()]]

In [ ]:
df.iloc[[df['ddg'].idxmax()]]

In [7]:
# save processed dataset
import torch

df.to_csv("./data/info.csv",index=False)

torch.save(df, "./data/dataset.pt")
info = torch.load("./data/dataset.pt")


In [ ]:
# collect .pdb file
data_pdb_path = "./data/pdb/"
os.makedirs(data_pdb_path, exist_ok=True)
os.system(f"cp datasets/skempi_v2/PDBs/*.pdb {data_pdb_path}")